In [88]:
import pandas as pd
import time
import sys
import os
import re


nombre_target = "../csv/eduroam/Eduroam_08112022.csv"

datos_edu = pd.read_csv(nombre_target, sep=',',on_bad_lines="skip")


In [89]:
def arreglarMAC(oidObject):
    oidObject = oidObject[-14:-2]
    return oidObject[0:2] + ':' + oidObject[2:4] + ':' + oidObject[4:6] + ':' + oidObject[6:8] + ':' + oidObject[8:10] + ':' + oidObject[10:12] 


def arreglo_df_Wifi_A(dataframe_Wifi_A):

    # Creación de la lista de IP de APs que nos interesan
    lista_nombre_APs = ['ANT-PB-N-O-Biblio', 'ANT-PB-N-C-Biblio', 'ANT-PB-N-E-Biblio', 'ANT-PB-E-NN-Biblio', 'ANT-PB-E-N-Biblio', 'ANT-PB-E-S-Biblio', 'ANT-PB-E-SS-Biblio']
    lista_IPs_APs = ['192.168.89.189', '192.168.89.187', '192.168.89.111', '192.168.89.184', '192.168.89.181', '192.168.89.182', '192.168.89.249']
    lista_IPs_APs = dict(zip(lista_nombre_APs, lista_IPs_APs))

    lista_completa_APs = lista_IPs_APs
    lista_completa_APs = pd.Series(lista_completa_APs)

    dataframe_Wifi_A = dataframe_Wifi_A.copy()
    dataframe_Wifi_A.columns = ['Nº Secuencia', 'Fecha_Hora', 'MAC','MACWLAN', 'IP', 'IPAP', 'Nombre', 'sistema operativo', 'relacion SN', 'TXDataFr', 'TXDataBy', 'TXRetries', 'TXRate', 'RXDataFr', 'RXDataBy', 'RXRetries', 'RXRate']
    dataframe_Wifi_A.drop_duplicates(inplace=True)
    dataframe_Wifi_A = dataframe_Wifi_A[(dataframe_Wifi_A['MAC'] != '0') & (dataframe_Wifi_A['MAC'].notna()) & (dataframe_Wifi_A['MACWLAN'].notna()) & (dataframe_Wifi_A['IP'].notna()) & (dataframe_Wifi_A['IPAP'].notna())]

    # Código propio para arreglar las columnas de MACs e IPs respecto a la nomenclatura SNMP.
    dataframe_Wifi_A['MAC'] = [arreglarMAC(oid) for oid in dataframe_Wifi_A['MAC']]
    dataframe_Wifi_A['MACWLAN'] = [arreglarMAC(oid) for oid in dataframe_Wifi_A['MACWLAN']]

    # Código TFM, mediante expresiones regulares, para arreglar MACs e IPs respecto nomenclatura SNMP
    dataframe_Wifi_A['IP'] = dataframe_Wifi_A['IP'].str.extract(r" ([\d\.]+)']$")
    dataframe_Wifi_A['IPAP'] = dataframe_Wifi_A['IPAP'].str.extract(r" ([\d\.]+)']$")

    # Algunas MACs están corruptas, por lo que aplicamos esta expresión regular para filtrarlas
    # Fuente: https://stackoverflow.com/questions/7629643/how-do-i-validate-the-format-of-a-mac-address
    dataframe_Wifi_A['MAC'] = dataframe_Wifi_A['MAC'].apply(lambda x: x if re.match("[0-9a-f]{2}([-:]?)[0-9a-f]{2}(\\1[0-9a-f]{2}){4}$", x.lower()) else 0)
    dataframe_Wifi_A = dataframe_Wifi_A[dataframe_Wifi_A['MAC'] != 0]

    dataframe_Wifi_A = dataframe_Wifi_A[dataframe_Wifi_A['IPAP'].isin(lista_completa_APs)]
    dataframe_Wifi_A = dataframe_Wifi_A[~dataframe_Wifi_A["IP"].str.contains("192.168.102.")]

    return  dataframe_Wifi_A

In [90]:
datos_edu = arreglo_df_Wifi_A(datos_edu)
datos_edu.to_csv("../csv/eduroam/2022-11-08_eduroam.csv",index=False)

In [96]:
datos_edu[datos_edu['IP']=="192.168.245.225"]

,Nº Secuencia,Fecha_Hora,MAC,MACWLAN,IP,IPAP,Nombre,sistema operativo,relacion SN,TXDataFr,TXDataBy,TXRetries,TXRate,RXDataFr,RXDataBy,RXRetries,RXRate
722,117,2022-11-08 08:18:02,ba:02:5b:ab:2e:64,60:26:ef:6b:18:71,192.168.245.225,192.168.89.249,NaN,NOFP,56.0,52599.0,5897339.0,454.0,72.0,61095.0,12852780.0,232.0,72.0
5276,489,2022-11-08 09:00:01,b8:3b:cc:30:91:4f,cc:88:c7:07:e3:c2,192.168.245.225,192.168.89.249,NaN,NOFP,60.0,161455.0,16061910.0,17.0,150.0,234116.0,37977.0,38.0,72.0
6012,566,2022-11-08 09:03:22,b8:94:36:82:5e:d5,a8:5b:f7:b6:c5:c0,192.168.245.225,192.168.89.249,NaN,NOFP,42.0,193985.0,130870.0,46.0,52.0,3528.0,44117.0,40.0,72.0


In [94]:
datos_edu[datos_edu['IP']=="192.168.245.137"]

,Nº Secuencia,Fecha_Hora,MAC,MACWLAN,IP,IPAP,Nombre,sistema operativo,relacion SN,TXDataFr,TXDataBy,TXRetries,TXRate,RXDataFr,RXDataBy,RXRetries,RXRate
738,133,2022-11-08 08:18:02,dc:b7:2e:5b:00:ce,60:26:ef:69:b4:01,192.168.245.137,192.168.89.184,NaN,NOFP,25.0,71436.0,7264686.0,48.0,150.0,82204.0,14494383.0,100.0,200.0
1647,181,2022-11-08 08:33:02,dc:a6:32:da:67:f8,cc:88:c7:08:dd:52,192.168.245.137,192.168.89.184,NaN,NOFP,25.0,103062.0,9900960.0,76.0,150.0,118205.0,20836292.0,145.0,200.0
2176,242,2022-11-08 08:39:02,dc:a6:32:da:67:f8,cc:88:c7:08:dd:52,192.168.245.137,192.168.89.184,NaN,NOFP,24.0,115742.0,242569644.0,38061.0,78.0,117.0,8012.0,78.0,6.0
3644,387,2022-11-08 08:51:13,dc:b7:2e:5b:00:ce,cc:88:c7:08:dd:52,192.168.245.137,192.168.89.184,Raspberry8,NOFP,28.0,1103789.0,242792459.0,38106.0,390.0,14.0,322.0,6.0,1.0
4139,430,2022-11-08 08:54:02,dc:b7:2e:5b:00:ce,60:26:ef:69:b4:01,192.168.245.137,192.168.89.184,NaN,NOFP,25.0,1104048.0,242857688.0,38126.0,130.0,21.0,2270.0,3.0,6.0
4705,496,2022-11-08 08:57:19,dc:a6:32:da:67:f8,cc:88:c7:08:dd:52,192.168.245.137,192.168.89.184,NaN,NOFP,28.0,1.0,70.0,2.0,130.0,14.0,322.0,6.0,6.0
6910,705,2022-11-08 09:06:01,dc:a6:32:da:67:f8,cc:88:c7:08:dd:52,192.168.245.137,192.168.89.184,NaN,NOFP,25.0,206046.0,18198154.0,159.0,150.0,3959700.0,2270.0,6.0,1.0
7762,735,2022-11-08 09:09:06,dc:a6:32:da:67:f8,cc:88:c7:08:dd:52,192.168.245.137,192.168.89.182,Raspberry8,NOFP,30.0,1108823.0,243938670.0,38262.0,325.0,3963454.0,364332550.0,29155.0,1.0
10378,749,2022-11-08 09:18:27,dc:b7:2e:5b:00:ce,cc:88:c7:08:dd:52,192.168.245.137,192.168.89.184,NaN,NOFP,24.0,246636.0,21432187.0,240.0,150.0,344740.0,67283852.0,474.0,390.0
18146,740,2022-11-08 09:45:02,dc:b7:2e:5b:00:ce,cc:88:c7:08:dd:52,192.168.245.137,192.168.89.184,NaN,NOFP,25.0,326953.0,27925078.0,397.0,150.0,491073.0,98253934.0,1180.0,200.0


In [95]:
a = datos_edu[datos_edu['MACWLAN']!="cc:88:c7:08:dd:52"]
a = a[a["Nombre"]=="Raspberry8"]
a

,Nº Secuencia,Fecha_Hora,MAC,MACWLAN,IP,IPAP,Nombre,sistema operativo,relacion SN,TXDataFr,TXDataBy,TXRetries,TXRate,RXDataFr,RXDataBy,RXRetries,RXRate
5350,563,2022-11-08 09:00:01,dc:fb:48:87:61:99,cc:88:c7:08:e7:51,192.168.248.64,192.168.89.184,Raspberry8,NOFP,30.0,29.0,384.0,1.0,270.0,10328.0,5270317.0,197.0,135.0
6098,652,2022-11-08 09:03:22,de:0c:40:0e:9f:d6,60:26:ef:69:4a:91,192.168.241.123,192.168.89.184,Raspberry8,Android,15.0,29.0,293794.0,1.0,0.0,0.0,2070981.0,105.0,300.0
24558,806,2022-11-08 10:06:01,dc:1b:a1:d3:d7:18,60:26:ef:67:73:c0,192.168.250.20,192.168.89.182,Raspberry8,NOFP,28.0,1117722.0,245734095.0,38607.0,117.0,3999692.0,367438264.0,30037.0,390.0
26492,822,2022-11-08 10:12:02,dc:a6:32:64:a2:6b,cc:88:c7:09:c5:92,192.168.63.39,192.168.89.182,Raspberry8,Android,16.0,8.0,848.0,102.0,39.0,5.0,760.0,2.0,39.0
31399,832,2022-11-08 10:39:02,dc:b7:2e:5b:00:ce,60:26:ef:69:b4:11,192.168.252.173,192.168.89.182,Raspberry8,NOFP,42.0,858.0,658150.0,53.0,54.0,1258.0,369538175.0,30628.0,390.0
32400,853,2022-11-08 10:45:01,dc:b7:2e:5b:00:ce,cc:88:c7:09:c5:92,192.168.252.173,192.168.89.182,Raspberry8,NOFP,29.0,1124390.0,247058739.0,512.0,150.0,848721.0,167823307.0,13500.0,433.0
36477,881,2022-11-08 10:57:02,dc:b7:2e:5b:00:ce,60:26:ef:69:b4:11,192.168.245.137,192.168.89.182,Raspberry8,NOFP,40.0,44.0,8815593.0,583.0,650.0,23431.0,0.0,0.0,0.0
38586,912,2022-11-08 11:03:02,de:0b:44:75:c0:13,cc:88:c7:08:d0:51,192.168.252.125,192.168.89.184,Raspberry8,NOFP,29.0,1126728.0,247616068.0,38948.0,292.0,4037795.0,370832784.0,30978.0,390.0
39660,920,2022-11-08 11:06:27,dc:b7:2e:61:68:c0,60:26:ef:69:b4:11,192.168.61.195,192.168.89.184,Raspberry8,Linux,29.0,44.0,45411522.0,66.0,300.0,23431.0,4146407.0,168.0,0.0
41915,970,2022-11-08 11:18:02,dc:b7:2e:5b:00:ce,60:26:ef:69:b4:11,192.168.61.195,192.168.89.182,Raspberry8,NOFP,30.0,1135197.0,20951177.0,429.0,65.0,10244.0,527089.0,924.0,57.0


In [117]:
datos_edu[datos_edu['MAC'] == "dc:a6:32:da:67:f8"]

,Nº Secuencia,Fecha_Hora,MAC,MACWLAN,IP,IPAP,Nombre,sistema operativo,relacion SN,TXDataFr,TXDataBy,TXRetries,TXRate,RXDataFr,RXDataBy,RXRetries,RXRate
1647,181,2022-11-08 08:33:02,dc:a6:32:da:67:f8,cc:88:c7:08:dd:52,192.168.245.137,192.168.89.184,NaN,NOFP,25.0,103062.0,9900960.0,76.0,150.0,118205.0,20836292.0,145.0,200.0
2176,242,2022-11-08 08:39:02,dc:a6:32:da:67:f8,cc:88:c7:08:dd:52,192.168.245.137,192.168.89.184,NaN,NOFP,24.0,115742.0,242569644.0,38061.0,78.0,117.0,8012.0,78.0,6.0
4705,496,2022-11-08 08:57:19,dc:a6:32:da:67:f8,cc:88:c7:08:dd:52,192.168.245.137,192.168.89.184,NaN,NOFP,28.0,1.0,70.0,2.0,130.0,14.0,322.0,6.0,6.0
6910,705,2022-11-08 09:06:01,dc:a6:32:da:67:f8,cc:88:c7:08:dd:52,192.168.245.137,192.168.89.184,NaN,NOFP,25.0,206046.0,18198154.0,159.0,150.0,3959700.0,2270.0,6.0,1.0
7762,735,2022-11-08 09:09:06,dc:a6:32:da:67:f8,cc:88:c7:08:dd:52,192.168.245.137,192.168.89.182,Raspberry8,NOFP,30.0,1108823.0,243938670.0,38262.0,325.0,3963454.0,364332550.0,29155.0,1.0
20828,769,2022-11-08 09:54:01,dc:a6:32:da:67:f8,cc:88:c7:08:dd:52,192.168.245.137,192.168.89.184,NaN,NOFP,25.0,351703.0,29927484.0,414.0,150.0,541234.0,108049671.0,2919.0,200.0
23609,805,2022-11-08 10:03:34,dc:a6:32:da:67:f8,cc:88:c7:08:dd:52,192.168.245.137,192.168.89.184,NaN,NOFP,31.0,4031.0,3453392.0,473.0,19.0,5379.0,920717.0,933.0,104.0
25521,816,2022-11-08 10:09:34,dc:a6:32:da:67:f8,cc:88:c7:08:dd:52,192.168.252.173,192.168.89.182,Raspberry8,NOFP,30.0,1119462.0,245949330.0,38647.0,292.0,4003635.0,367887340.0,30108.0,433.0
34417,861,2022-11-08 10:51:01,dc:a6:32:da:67:f8,cc:88:c7:09:c5:92,192.168.245.137,192.168.89.182,TC14,NOFP,29.0,1124884.0,247211280.0,38877.0,325.0,4029931.0,1655948.0,809.0,650.0
44205,975,2022-11-08 11:24:02,dc:a6:32:da:67:f8,cc:88:c7:08:dd:52,192.168.245.137,192.168.89.184,NaN,NOFP,29.0,1136389.0,4243754.0,14.0,300.0,34230.0,5841359.0,749.0,300.0


In [107]:
lista_n = datos_edu[~datos_edu['Nombre'].isna()]['Nombre'].drop_duplicates()
lista_n

10        ja.delolmofernandez@um.es
18                22952066K@upct.es
21                 22996267@upct.es
31                22997957G@upct.es
32                29154796L@upct.es
                    ...            
172212                     23952810
172232                     48741022
172258           l.santosmeca@um.es
172291            realme-GT-Neo2-5G
172379                    24427754Y
Name: Nombre, Length: 958, dtype: object

In [108]:
for i in lista_n:
    datos_edu[datos_edu['Nombre'] == i]
    print(i)

ja.delolmofernandez@um.es
22952066K@upct.es
22996267@upct.es
22997957G@upct.es
29154796L@upct.es
27450941@upct.es
anonymous@upct.es
22952309B@upct.es
27437095@upct.es
29154796
anonymous@um.es
55536172G
23812428E
Galaxy-S9
OnePlus-Nord2-5G
aya.amjid@um.es
31656066Q
77577488@upct.es
realme-8
27436743c
54565032Q
lorena.gimenom@um.es
meganlouise.kestell@um.es
23954723Q
miriam.gomezg@um.es
DESKTOP-D66SPI6
24417199@upct.es
Redmi-Note-8T
23958521
23303508T
48635997
realme-C35
23954723
58469142
anonymous_20180412@um.es
24417032W
48853696V
77519502@upct.es
Galaxy-A5-2016
27485320@upct.es
26649110E
24417329
anonymous_20181026@um.es
Raspberry8
74028928R
a.perezmoreno@um.es
22940810n
jesus.jimenezs@um.es
m.requenavalencia@um.es
23956333Q
35693616p
77836387V
77840837@upct.es
23832738T
48853233z
HUAWEI_P_smart_2019-6230d
22986970b
22967160G
52805598J@upct.es
23954896m
48837382x
raspberrypi
49476834
Y6153336
HUAWEI_P20_lite-37dbfa7a7
48750231y
pedro.v.g@um.es
M2003J15SC-RedmiNote
DESKTOP-0OO6GBC
2331

In [111]:
l_m = datos_edu[datos_edu['Nombre'] == "27436743c"]["MAC"].drop_duplicates()

In [113]:
l_m

1926      f6:08:04:1c:72:5c
2515      f4:7d:ef:4d:10:80
3691      f4:96:34:6c:65:fb
74891     f2:9f:f6:b6:e3:a3
86345     f2:e6:6a:0c:05:27
88177     f2:af:3b:4a:b9:30
104844    f4:60:e2:8e:a4:12
105492    f4:63:1f:84:c2:03
Name: MAC, dtype: object

In [114]:
datos_edu[datos_edu['MAC'].isin(l_m)]

,Nº Secuencia,Fecha_Hora,MAC,MACWLAN,IP,IPAP,Nombre,sistema operativo,relacion SN,TXDataFr,TXDataBy,TXRetries,TXRate,RXDataFr,RXDataBy,RXRetries,RXRate
92,93,2022-11-08 08:00:02,f2:e6:6a:0c:05:27,cc:88:c7:08:64:d1,192.168.244.176,192.168.89.181,29154796,NOFP,40.0,7940.0,4921700.0,101.0,270.0,13839.0,1075409.0,114.0,300.0
199,101,2022-11-08 08:03:01,f2:e6:6a:0c:05:27,cc:88:c7:08:64:d1,192.168.244.176,192.168.89.181,29154796,NOFP,39.0,8041.0,4994736.0,101.0,300.0,13950.0,1091970.0,114.0,300.0
312,107,2022-11-08 08:06:01,f2:e6:6a:0c:05:27,cc:88:c7:08:64:d1,192.168.244.176,192.168.89.181,29154796,NOFP,38.0,8055.0,4995789.0,101.0,270.0,13999.0,1094597.0,115.0,243.0
452,133,2022-11-08 08:12:01,f2:e6:6a:0c:05:27,cc:88:c7:08:64:d1,192.168.244.176,192.168.89.181,29154796,NOFP,39.0,8067.0,4996605.0,101.0,270.0,14035.0,1095581.0,115.0,270.0
598,140,2022-11-08 08:15:01,f2:e6:6a:0c:05:27,cc:88:c7:08:64:d1,192.168.244.176,192.168.89.181,29154796,NOFP,39.0,8073.0,4997013.0,101.0,270.0,14053.0,1096073.0,115.0,234.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175534,68,2022-11-08 22:42:02,f2:e6:6a:0c:05:27,cc:88:c7:08:64:d1,192.168.244.176,192.168.89.181,29154796,NOFP,38.0,12309.0,7012082.0,229.0,270.0,22719.0,1699584.0,268.0,270.0
175607,70,2022-11-08 22:45:02,f2:e6:6a:0c:05:27,cc:88:c7:08:64:d1,192.168.244.176,192.168.89.181,29154796,NOFP,38.0,12315.0,7012490.0,229.0,300.0,22737.0,1700076.0,268.0,300.0
175681,71,2022-11-08 22:48:02,f2:e6:6a:0c:05:27,cc:88:c7:08:64:d1,192.168.244.176,192.168.89.181,29154796,NOFP,38.0,12321.0,7012898.0,229.0,300.0,22755.0,1700568.0,268.0,300.0
175753,69,2022-11-08 22:51:02,f2:e6:6a:0c:05:27,cc:88:c7:08:64:d1,192.168.244.176,192.168.89.181,29154796,NOFP,39.0,12327.0,7013306.0,229.0,270.0,22773.0,1701060.0,268.0,270.0


In [115]:
ns = datos_edu[datos_edu['MAC'].isin(l_m)]['Nombre'].drop_duplicates()
ns

92                        29154796
1924                      realme-8
1926                     27436743c
2209               aya.amjid@um.es
2856                      58469142
6171      anonymous_20181026@um.es
9592           marta.prietop@um.es
10461                    24421911m
14789            58477224t@upct.es
14793                    49475625A
34519            anonymous@upct.es
43172                   Hiba-s-A22
56931                    77701878N
86342                   Galaxy-A10
87268                    24408724C
87271                    24408680E
106160                    49335017
108858                   54805853G
110199    realme-GT-Master-Edition
Name: Nombre, dtype: object

In [119]:
algo = [None] * 2
algo

[None, None]